# Summary

**Goal: Calculate peptide distribution and protein distribution P(Zj)** from P(Xi|Zj) fluorosequencing scores 

___

**Prior to my code**

1. **Acquiring of experimental data (reads)**
    - **4-5 fluorescence level plots vs Edman degradation cycle, 1 read for each molecule**


2. **Matts classifier converts reads into P(Xi|Zj)**, the likelihood of each partial sequence given a peptide.
    - **Calculating P(Xi) directly impossible/difficult**. You may think that it is simply number of partial sequences/number of all reads, but that is only true if the number of reads is much larger than the number of possible outcomes. It's like rolling a die a couple times, and then asking how likely each number is. You can only do that if you are certain that your number of rolls adequately samples the possibilities. In our case, we do not know this and thus it is instead more consistent to **calculate P(Xi|Zj) instead and to use that to calculate P(Zj), the target variable.**
    
___

**This code**

1. Calculation of peptide distribution (P(Zj)) using P(Xi|Zj) scores
    1. **imports P(Xi|Zj) table, generated by Matt's program**
    2. Asks user for settings
    3. For each bootstrap run:
        1. Create a subtable created from randomly sampling rows in P(Xi|Zj)
        2. Use this subtable of P(Xi|Zj) to run EM until P(Zj) converges. P(zj) calculation is like Monte Carlo integration, because P(Zj|Xi)  \
           **For each EM run:**
            
           - **Hen, egg problem. two missing variables: P(Zj|Xi) and P(Zj). calculating either requires the other**
           - for **1st calculation of P(Zj|Xi) approximate: assume equal distribution of all P(Zj)**
           - **alternate between updating P(Zj|Xi) and P(Zj) until convergence**
                
    4. Calculate **average P(Zi) from all bootstrap runs, and calculate 95% CI**
    
2. Calculation of protein distribution using calculated peptide distribution
    1. **Calculate P(Xi|Zj) from number of matches against digested proteome**
    1. run **bootstrap-EM, like in peptide inferring step**
        - First calculation of P(Zj|Xi) assumes equal distribution of all P(Zj)
        - **Uses peptide distribution P(Zj) values from (1) as P(Xi), which simplifies the Bayes-derived equation.** However, the principle is the same
        
___
**Open questions**
- Is there a case in which bootstrapping improves the guess? Test dataset with many rare peptides
- make protein inferring work
- suggestions for data I should be showing?
    - plot of results depending on number of EM runs, compared to argmax and weighted score
    - show effect of bootstrap runs and see if its advantageous to use that for any dataset
    - both for peptide and for protein inferring

# Functions

## import of packages

In [171]:
import numpy as np # better arrays than inbuilt arrays
import matplotlib.pyplot as plt # to plot stuff

import pandas as pd #for DataFrame tables
from IPython.display import display #to display dfs more nicely. works similar to head(), but is more flexible in how many columns/rows are shown
pd.set_option("display.max_rows", None) #to let display show full df
pd.set_option("display.max_columns", None)

import scipy.stats
#from scipy.stats import norm
import statistics

import math
import time #to measure runtime

## EM    

In [172]:
def update_p_zj_given_xi(p_xi_given_zj, p_zj, p_zj_given_xi):
    ## for PEPTIDE inference
    if inference_mode == "peptide" or "protein":
        numerator = p_xi_given_zj * p_zj # = ARRAY with same shape as P(Xi|Zj)
        denominator = np.expand_dims(np.sum(p_xi_given_zj * p_zj, axis=1), axis=-1) # = VECTOR with length of p Xi. expand_dims is necessary because else denominator has the shape (number of reads Xi,) instead of (number of reads Xi, 1). The entries of the vector are identical. But it does not have the same matrix "rank", which is why broadcasting fails without that 
        p_zj_given_xi = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0) # if denominator = 0, return 0. This happens sometimes when the likelihood for Zj is equal to 0. This could alternatively be avoided by cropping the rows where the input P(Xi|Zj) scores equal to 0
        p_zj_given_xi = np.transpose(p_zj_given_xi)
       
    ## for PROTEIN inference (denominator is has no loop)
    elif inference_mode == "not protein - doesn't work for some reason, even though I expect it to...?":
        p_zj_given_xi = (p_xi_given_zj * p_zj)/np.expand_dims(p_xi, axis=-1)
        p_zj_given_xi = np.transpose(p_zj_given_xi)
        
    else:
        print("ERROR: inference_mode was not set to peptide or protein")
        
    return p_zj_given_xi

In [173]:
def update_p_zj(p_zj, p_zj_given_xi):  
    if inference_mode == "peptide":
        p_zj = p_zj_given_xi.sum(axis=1)/p_xi_given_zj.shape[1]

        p_zj_sum = np.sum(p_zj, axis = 0)
        p_zj = p_zj/p_zj_sum

    elif inference_mode == "protein":
        p_zj.fill(0) # reset all values of pzj to 0
        
        p_zj = np.sum(p_zj_given_xi * p_xi, axis = 1)
        p_zj_sum = np.sum(p_zj, axis = 0)
        p_zj = p_zj/p_zj_sum    
    
    else:
        print("ERROR: inference_mode was not set to peptide or protein")
    
    return p_zj

In [174]:
def EM_convergence_checker(p_zj_old, p_zj, EM_convergence_minimum):
    difference_abs = abs(np.sum(p_zj_old[round(0.5*len(p_zj_old))::], axis=0, dtype=float) - np.sum(p_zj[round(0.5*len(p_zj_old))::], axis=0, dtype=float))
    if difference_abs > EM_convergence_minimum:
        return False
    
    else:
        return True   

In [175]:
def EM(p_xi_given_zj):
    ### initialise p_zj_given_xi
    # p_xi_given_zj = np.arange(0,102,1).reshape(34,3) # for testing EM iterating through columns and rows
    p_zj_given_xi = np.full((p_xi_given_zj.shape[1], p_xi_given_zj.shape[0]), 0, dtype=float) #Initialisation based on array size of p_xi_given_zj -- same size, but transposed
    
    ### initialise p_zj
    n = p_xi_given_zj.shape[1] # number of peptides
    p_zj_initial = 1/n #initial approximation: all zj equally likely, to jumpstart first iteration

    global p_zj
    p_zj = np.full(n, p_zj_initial)
    p_zj_old = np.full(n, 0)
    
    global EM_loopcounter 
    EM_loopcounter = 0
    EM_convergence_checker_result = False
        
    while (EM_convergence_checker_result == False) and EM_loopcounter < EM_loopcounter_max:
        p_zj_given_xi = update_p_zj_given_xi(p_xi_given_zj, p_zj, p_zj_given_xi)
        
        p_zj_old = np.copy(p_zj, order='K', subok=False)
        p_zj = update_p_zj(p_zj, p_zj_given_xi)
        EM_convergence_checker_result = EM_convergence_checker(p_zj_old, p_zj, EM_convergence_minimum)

        EM_loopcounter = EM_loopcounter + 1
    return p_zj

## bootstrapping

**create_subarray_of_p_xi_given_zj(p_xi_given_zj)**

   - Returns subarray of P(Xi|Zj) input data. 
   - Subarray's size is a fraction of the full datasize, this fraction is user-specified (default: 0.7). Sampling is done "with replacement".
   
**bootstrap_EM()**

   - returns: P(Zj) values after bootstrap/EM, in relative values (i. e. in numbers <= 1)
   - if bootstrap_sampled_fraction is set to -1, no bootstrapping occurs, and just one EM using full P(Xi|Zj) dataset is run
   - else, multiple bootstraps are run (default: 200), in which each EM uses only a subarray created from the full dataset using the create_subarray_of_p_xi_given_zj function
   
**bootstrap_EM_analytics_AVG():**

   - returns: the average P(Zj) from all bootstrap runs
   
**bootstrap_EM_analytics_CI(CI_percent)**

   - returns: confidence interval of all bootstrap runs (default: 95% CI)

In [176]:
def create_subarray_of_p_xi_given_zj(p_xi_given_zj):
    df_p_xi_given_zj = pd.DataFrame(p_xi_given_zj)
    df_p_xi_given_zj_sample = df_p_xi_given_zj.sample(frac=bootstrap_sampled_fraction, axis='rows', replace=True) # filters for a random partial dataset
    # display(pd.DataFrame(df_p_xi_given_zj_sample))
    p_xi_given_zj_subarray = df_p_xi_given_zj_sample.to_numpy()
    
    # print("p_xi_given_zj_subarray")
    # display(pd.DataFrame(p_xi_given_zj_subarray))
    
    return p_xi_given_zj_subarray

In [177]:
def bootstrap_EM():
    i = 0
    
    p_zj_bootstrap_results_absolute = np.full((p_xi_given_zj.shape[1]), 0, dtype=float)
      
    if bootstrap_sampled_fraction == -1: # no bootstrapping, i. e. use full dataset for EM. Since the EM is deterministic, there is no point in running multiple bootstrap runs
        p_xi_given_zj_subarray = p_xi_given_zj
        p_zj_bootstrap_results_absolute = EM(p_xi_given_zj_subarray)
    
    else: # with bootstrapping, i. e. a subarray of the dataset is created for each bootstrap run
        while i < n_bootstrap_runs:
            p_xi_given_zj_subarray = create_subarray_of_p_xi_given_zj(p_xi_given_zj) 
            if i == 0:
                p_zj_bootstrap_results_absolute = EM(p_xi_given_zj_subarray)
            else:
                p_zj_bootstrap_results_absolute = np.vstack((p_zj_bootstrap_results_absolute, EM(p_xi_given_zj_subarray)))
            
            print("Bootstrap run #", i, ". EM loops: ", EM_loopcounter, sep="")
            if EM_loopcounter == EM_loopcounter_max:
                print("WARNING: EM_loopcounter was reached, convergence likely still has not been reached. Consider increasing the maximum number of EM loops.")

            i = i + 1
    
    #print("P_zj_bootstrap_results_absolute", p_zj_bootstrap_results_absolute)
    
    global p_zj_bootstrap_results_fraction
    if bootstrap_sampled_fraction == -1:
         p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 0)
    else:
        p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 1)[0]
        
    if inference_mode == "peptide":
        print("Peptide Zj values from every bootstrap run (columns: peptides, rows: bootstrap run, displayed as fractions: ")
    elif inference_mode == "protein":
        print("PRE-CORRECTION: Protein Zj values from every bootstrap run (columns: protein, rows: bootstrap run, displayed as fractions: ")
    display(pd.DataFrame(p_zj_bootstrap_results_fraction))
    return

In [178]:
def bootstrap_EM_protein():    
    if bootstrap_sampled_fraction == -1:
        bootstrap_EM()
        
    else:
        i = 0
        global p_zj_bootstrap_results_fraction
        peptide_p_zj_bootstrap_results_fraction = np.copy(p_zj_bootstrap_results_fraction)
        p_zj_bootstrap_results_absolute = np.full((p_xi_given_zj.shape[1]), 0, dtype=float)
        for row in peptide_p_zj_bootstrap_results_fraction:
            p_xi_given_zj_subarray = p_xi_given_zj
            p_xi = np.array(row)
            p_zj_bootstrap_results_absolute = np.vstack((p_zj_bootstrap_results_absolute, EM(p_xi_given_zj_subarray)))

        if bootstrap_sampled_fraction == -1:
             p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 0)
        else:
            p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 1)[0]

        print("PRE-CORRECTION: Protein Zj values from every bootstrap run (columns: protein, rows: bootstrap run, displayed as fractions: ")
        display(pd.DataFrame(p_zj_bootstrap_results_fraction))
    return

In [179]:
def bootstrap_EM_analytics_AVG():
    global p_zj_bootstrap_results_fraction_avg
    
    if bootstrap_sampled_fraction == -1:
        p_zj_bootstrap_results_fraction_avg = np.copy(p_zj_bootstrap_results_fraction,order='K')
    else:
        p_zj_bootstrap_results_fraction_avg = np.sum(p_zj_bootstrap_results_fraction, axis = 0)/p_zj_bootstrap_results_fraction.shape[0]

In [180]:
def bootstrap_EM_analytics_CI():
    highindex = round(CI_percent*len(p_zj_bootstrap_results_fraction))
    lowindex = round((1-CI_percent)*len(p_zj_bootstrap_results_fraction))
    
    bootstrap_CI_max = np.sort(p_zj_bootstrap_results_fraction, axis=0, kind="quicksort", order=None)[lowindex:highindex:][-1] #fetches last row from sorted table, ie the highest values. axis=0: sorting along each column
    bootstrap_CI_min = np.sort(p_zj_bootstrap_results_fraction, axis=0, kind="quicksort", order=None)[lowindex:highindex:][0] #same but for first row, highest values
    bootstrap_CI_minmax = np.stack((bootstrap_CI_min, bootstrap_CI_max))
    
    ### show full sorted table
    # global p_zj_bootstrap_results_fraction_CI
    # print("The", CI_percent*100, "confidence interval of all zj bootstrapping values sorted, displayed as fractions: ")    
    # p_zj_bootstrap_results_fraction_CI = np.sort(p_zj_bootstrap_results_fraction, axis=0, kind="quicksort", order=None)[lowindex:highindex:] #axis=sorting along each column
    # display(pd.DataFrame(p_zj_bootstrap_results_fraction_CI))
    
    return bootstrap_CI_minmax

## output

In [181]:
def output():
    global p_zj_bootstrap_results_fraction_avg
    bootstrap_EM_analytics_AVG()
    bootstrap_CI_minmax = bootstrap_EM_analytics_CI()

    ### STDs (alternative to CI)
    p_zj_bootstrap_results__fraction_std = np.std(p_zj_bootstrap_results_fraction, axis = 0)

    if inference_mode == "protein":
        p_zj_bootstrap_results_fraction_avg = p_zj_bootstrap_results_fraction_avg/dyeseq_in_prot_match_count / np.sum(p_zj_bootstrap_results_fraction_avg/dyeseq_in_prot_match_count)
                
    # Combines AVG zj values (from all bootstrap runs), the STD, and the two bounds for the user-chosen confidence interval in one  
    if bootstrap_sampled_fraction == -1: #ie if no bootstrapping. Fills -CI and +CI columns with "N/A" values
        p_zj_avg_plus_CI = np.stack((p_zj_bootstrap_results_fraction_avg*100, np.full((len(p_zj_bootstrap_results_fraction_avg)), fill_value="N/A", dtype=None), np.full((len(p_zj_bootstrap_results_fraction_avg)), fill_value="N/A", dtype=None), np.full((len(p_zj_bootstrap_results_fraction_avg)), fill_value="N/A", dtype=None)), axis=1)    
        display(pd.DataFrame(p_zj_avg_plus_CI, columns = ["AVG [%]", "±STD [%]", "-CI [%]", "+CI [%]"]).astype({"AVG [%]":float}).round(4))
    else:
        p_zj_avg_plus_CI = np.stack((p_zj_bootstrap_results_fraction_avg, p_zj_bootstrap_results__fraction_std, bootstrap_CI_minmax[0], bootstrap_CI_minmax[1]), axis = 1)
        display(pd.DataFrame(p_zj_avg_plus_CI, columns = ["AVG [%]", "±STD [%]", "-CI [%]", "+CI [%]"]).round(4)*100)

    ### print user-settings
    print('\033[1m' + 'Settings of this run' + '\033[0m')
    print("Inference mode:", inference_mode)
    print("EM_convergence_minimum:", EM_convergence_minimum)
    print("EM_loopcounter_max:", EM_loopcounter_max)
    print("bootstrap_sampled_fraction:", bootstrap_sampled_fraction)
    print("n_bootstrap_runs:", n_bootstrap_runs)
    print("CI_percent:", CI_percent, "\n")

    ### print other
    print("Runtime: %s seconds ---" % round((time.time() - start_time),2)) #run time

# call functions

In [182]:
# esc, ctrl+a, ctrl+enter to run all cells

#p_xi_given_zj = create_3_gaussians_and_calculate_p_xi_given_zj(140, 150, 160, 15, 15, 15, 5, 5, 5) #mean1-3, std1-3, n1-3
# plot_histograms_and_pdfs_from_gaussians(140, 150, 160, 15, 15, 15, 5, 5, 5)

EM_convergence_minimum = float(input("EM_convergence_minimum? If nothing is entered, it is set to 0.0001.") or "0.0001")
EM_loopcounter_max = int(input("Maximum number of EM runs (per bootstrap run)? If nothing is entered, it is set to 200.") or "200") 
bootstrap_sampled_fraction = float(input("Fraction of subarray sampled for each bootstrap run? If nothing is entered, it is set to 0.8. If -1 is entered, bootstrapping is turned off. Note: Bootstrapping is always turned OFF for the protein inference part.") or "0.01")                   
n_bootstrap_runs = int(input("Number of bootstrap runs? If nothing is entered, it is set to 200.") or "200") 
CI_percent = float(input("Condidence interval? If nothing is entered, it is set to 95.") or "95")

start_time = time.time() # to start measuring runtime
p_xi_given_zj = np.genfromtxt('../110 data set/110-peps-shuffled-scores.csv', delimiter=',') # import P(Xi|Zj) table (on the read level - i. e. Likelihood of any given read Xi assuming a peptide Zj), calculated by Matt

EM_convergence_minimum? If nothing is entered, it is set to 0.0001. 
Maximum number of EM runs (per bootstrap run)? If nothing is entered, it is set to 200. 
Fraction of subarray sampled for each bootstrap run? If nothing is entered, it is set to 0.8. If -1 is entered, bootstrapping is turned off. Note: Bootstrapping is always turned OFF for the protein inference part. 
Number of bootstrap runs? If nothing is entered, it is set to 200. 
Condidence interval? If nothing is entered, it is set to 95. 


In [183]:
inference_mode = "peptide"
bootstrap_EM()
output()

Bootstrap run #0. EM loops: 61
Bootstrap run #1. EM loops: 58
Bootstrap run #2. EM loops: 48
Bootstrap run #3. EM loops: 24
Bootstrap run #4. EM loops: 42
Bootstrap run #5. EM loops: 48
Bootstrap run #6. EM loops: 33
Bootstrap run #7. EM loops: 38
Bootstrap run #8. EM loops: 37
Bootstrap run #9. EM loops: 43
Bootstrap run #10. EM loops: 28
Bootstrap run #11. EM loops: 34
Bootstrap run #12. EM loops: 28
Bootstrap run #13. EM loops: 28
Bootstrap run #14. EM loops: 21
Bootstrap run #15. EM loops: 24
Bootstrap run #16. EM loops: 25
Bootstrap run #17. EM loops: 47
Bootstrap run #18. EM loops: 31
Bootstrap run #19. EM loops: 49
Bootstrap run #20. EM loops: 49
Bootstrap run #21. EM loops: 38
Bootstrap run #22. EM loops: 55
Bootstrap run #23. EM loops: 38
Bootstrap run #24. EM loops: 44
Bootstrap run #25. EM loops: 19
Bootstrap run #26. EM loops: 45
Bootstrap run #27. EM loops: 41
Bootstrap run #28. EM loops: 36
Bootstrap run #29. EM loops: 29
Bootstrap run #30. EM loops: 36
Bootstrap run #31.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109
0,0.014155,0.032029,7.181205e-03,5.167138e-03,0.030181,2.656350e-03,0.009688,4.304371e-07,0.060279,1.892039e-02,7.171006e-03,2.133031e-03,4.126957e-03,9.682739e-12,0.012077,1.217962e-02,0.066318,1.371338e-02,4.645027e-03,2.830500e-02,7.708066e-04,4.252830e-06,6.130051e-03,0.036192,0.048742,7.184177e-06,0.021133,1.330776e-04,7.736915e-03,7.088425e-07,8.102066e-03,0.043507,1.142852e-03,1.042902e-06,1.898205e-07,0.000102,1.464063e-06,1.458908e-02,2.369290e-02,9.493708e-04,0.020676,0.021833,0.001689,1.478124e-05,5.906258e-03,2.238478e-03,1.720346e-07,0.009682,1.292689e-13,0.012548,8.195214e-03,0.014152,3.537929e-03,1.608222e-02,2.667023e-03,2.297125e-03,3.626134e-03,5.618575e-03,0.016643,9.136124e-03,1.983609e-02,1.058340e-03,7.495671e-06,1.327076e-03,4.935274e-04,9.956141e-03,5.833957e-07,2.038250e-04,2.500044e-03,3.250612e-03,0.019862,0.029493,3.420486e-06,3.194795e-08,2.503880e-03,4.095168e-04,7.314573e-07,4.897954e-21,0.001004,5.445740e-07,4.091608e-04,2.716224e-03,2.528553e-03,0.036954,0.023558,8.705243e-07,1.195675e-03,7.149180e-09,3.509137e-05,0.016626,2.342443e-02,4.358720e-11,1.234629e-03,3.461431e-04,3.515964e-04,3.251421e-03,1.454195e-04,0.053510,1.820138e-03,1.607305e-02,0.002678,1.977600e-08,1.012710e-06,6.875150e-12,3.217091e-03,5.828380e-04,0.016105,5.492814e-07,1.290251e-05,9.032362e-04
1,0.044123,0.099298,4.925506e-03,7.911907e-03,0.024284,3.491199e-03,0.011796,1.998143e-02,0.050895,1.785953e-02,8.860446e-03,1.366429e-03,1.067117e-02,9.141765e-03,0.006251,4.966506e-03,0.050036,1.587605e-05,8.683235e-03,3.424850e-08,2.087933e-03,8.025927e-11,8.443708e-03,0.002523,0.046135,3.893778e-09,0.024913,1.058926e-02,1.245588e-02,2.484638e-03,1.282720e-12,0.040725,1.243691e-04,7.969925e-04,4.014839e-03,0.001077,7.579400e-03,9.610222e-05,1.038892e-02,2.208596e-06,0.008981,0.014124,0.022612,3.221457e-05,3.940093e-06,3.602481e-05,4.743687e-08,0.003398,2.365444e-03,0.041124,1.593671e-04,0.030613,1.385067e-06,1.058807e-02,4.822104e-04,3.395432e-10,1.299059e-02,8.170692e-06,0.003522,1.782975e-09,6.712752e-04,8.439583e-08,1.013982e-04,9.472277e-06,4.405930e-03,3.314165e-03,2.205890e-03,1.896509e-08,1.012958e-06,2.035445e-05,0.051706,0.004224,1.169084e-03,4.021266e-05,4.607481e-07,6.101382e-03,4.234883e-04,3.471355e-04,0.005617,9.137586e-04,5.278456e-03,5.126019e-04,1.719011e-03,0.008708,0.040836,2.595435e-04,3.628072e-05,5.627785e-04,3.051089e-06,0.020979,5.350963e-03,2.382075e-04,4.343467e-03,9.253140e-05,4.276630e-03,6.494909e-05,4.648137e-07,0.025525,1.490522e-02,4.447343e-06,0.010642,1.674998e-03,4.171541e-05,1.597407e-05,4.585454e-03,1.585781e-02,0.039381,3.442771e-04,4.358326e-08,2.451024e-03
2,0.052695,0.027952,5.806272e-03,2.767110e-04,0.028398,1.410905e-11,0.006765,1.256883e-02,0.033054,1.407586e-02,1.143254e-02,4.597190e-07,1.277250e-02,3.935968e-05,0.022586,9.096963e-03,0.101016,6.249243e-03,2.671501e-03,2.970290e-03,1.730154e-03,3.798416e-08,1.646993e-02,0.030418,0.039018,1.933771e-11,0.013076,4.213060e-03,4.719192e-05,1.695783e-06,6.122272e-03,0.028503,1.491686e-07,2.511782e-04,7.141096e-07,0.000416,1.143596e-06,1.249541e-02,1.000968e-04,1.106399e-02,0.000846,0.033437,0.001524,1.080465e-02,1.765674e-04,2.016492e-06,2.096613e-04,0.008344,1.066915e-02,0.043351,3.355927e-09,0.048626,1.371479e-03,1.486943e-02,9.260593e-03,2.653893e-04,8.449586e-07,7.807663e-07,0.013833,6.860878e-03,3.187176e-03,8.035265e-03,1.350874e-02,4.247822e-08,7.162573e-06,4.901136e-04,6.473335e-03,2.786875e-08,1.384963e-13,2.976483e-04,0.030879,0.033470,2.385655e-05,3.999184e-04,1.261738e-06,1.205756e-05,9.796944e-03,3.772102e-09,0.014087,8.089582e-04,1.726058e-03,4.011904e-05,1.658536e-03,0.001539,0.031898,4.087583e-09,4.939

,AVG [%],±STD [%],-CI [%],+CI [%]
0,3.64,1.19,0.77,6.96
1,5.17,1.69,1.15,9.93
2,0.55,0.43,0.00,2.02
3,0.48,0.54,0.00,2.62
4,2.73,1.02,0.22,5.30
5,0.14,0.24,0.00,1.34
6,1.33,0.76,0.00,3.63
7,1.47,0.80,0.00,3.88
8,4.54,1.02,1.95,7.46
9,1.59,0.93,0.00,4.17


Settings of this run
Inference mode: peptide
EM_convergence_minimum: 0.0001
EM_loopcounter_max: 200
bootstrap_sampled_fraction: 0.01
n_bootstrap_runs: 200
CI_percent: 95.0 

Runtime: 17.27 seconds ---


In [184]:
inference_mode = "protein"

dyeseq_in_prot_match_count = np.genfromtxt("../110 data set/dyeseq_in_prot_match_count.csv", delimiter=',') # To calculate this the target proteome needs to be virtually digested and labelled (i.e. turned into dye sequences). Next, the reads' dye sequences are matched against this, and the number of matches per protein is counted. I did not write any code to solve this particular problem, but it seems like it should be fairly easy
p_xi_given_zj = np.genfromtxt("../110 data set/p_xi_given_zj prot_infer input.csv", delimiter=',') # The likelihood for any dye seq Xi given a protein Zj is equal to the number of matches of that dye seq in a particular protein, divided by the sum of all dye seq matches of that protein. I did not write any code to solve this problem either

display(pd.DataFrame(p_zj_bootstrap_results_fraction))

bootstrap_EM_protein()
output()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109
0,0.014155,0.032029,7.181205e-03,5.167138e-03,0.030181,2.656350e-03,0.009688,4.304371e-07,0.060279,1.892039e-02,7.171006e-03,2.133031e-03,4.126957e-03,9.682739e-12,0.012077,1.217962e-02,0.066318,1.371338e-02,4.645027e-03,2.830500e-02,7.708066e-04,4.252830e-06,6.130051e-03,0.036192,0.048742,7.184177e-06,0.021133,1.330776e-04,7.736915e-03,7.088425e-07,8.102066e-03,0.043507,1.142852e-03,1.042902e-06,1.898205e-07,0.000102,1.464063e-06,1.458908e-02,2.369290e-02,9.493708e-04,0.020676,0.021833,0.001689,1.478124e-05,5.906258e-03,2.238478e-03,1.720346e-07,0.009682,1.292689e-13,0.012548,8.195214e-03,0.014152,3.537929e-03,1.608222e-02,2.667023e-03,2.297125e-03,3.626134e-03,5.618575e-03,0.016643,9.136124e-03,1.983609e-02,1.058340e-03,7.495671e-06,1.327076e-03,4.935274e-04,9.956141e-03,5.833957e-07,2.038250e-04,2.500044e-03,3.250612e-03,0.019862,0.029493,3.420486e-06,3.194795e-08,2.503880e-03,4.095168e-04,7.314573e-07,4.897954e-21,0.001004,5.445740e-07,4.091608e-04,2.716224e-03,2.528553e-03,0.036954,0.023558,8.705243e-07,1.195675e-03,7.149180e-09,3.509137e-05,0.016626,2.342443e-02,4.358720e-11,1.234629e-03,3.461431e-04,3.515964e-04,3.251421e-03,1.454195e-04,0.053510,1.820138e-03,1.607305e-02,0.002678,1.977600e-08,1.012710e-06,6.875150e-12,3.217091e-03,5.828380e-04,0.016105,5.492814e-07,1.290251e-05,9.032362e-04
1,0.044123,0.099298,4.925506e-03,7.911907e-03,0.024284,3.491199e-03,0.011796,1.998143e-02,0.050895,1.785953e-02,8.860446e-03,1.366429e-03,1.067117e-02,9.141765e-03,0.006251,4.966506e-03,0.050036,1.587605e-05,8.683235e-03,3.424850e-08,2.087933e-03,8.025927e-11,8.443708e-03,0.002523,0.046135,3.893778e-09,0.024913,1.058926e-02,1.245588e-02,2.484638e-03,1.282720e-12,0.040725,1.243691e-04,7.969925e-04,4.014839e-03,0.001077,7.579400e-03,9.610222e-05,1.038892e-02,2.208596e-06,0.008981,0.014124,0.022612,3.221457e-05,3.940093e-06,3.602481e-05,4.743687e-08,0.003398,2.365444e-03,0.041124,1.593671e-04,0.030613,1.385067e-06,1.058807e-02,4.822104e-04,3.395432e-10,1.299059e-02,8.170692e-06,0.003522,1.782975e-09,6.712752e-04,8.439583e-08,1.013982e-04,9.472277e-06,4.405930e-03,3.314165e-03,2.205890e-03,1.896509e-08,1.012958e-06,2.035445e-05,0.051706,0.004224,1.169084e-03,4.021266e-05,4.607481e-07,6.101382e-03,4.234883e-04,3.471355e-04,0.005617,9.137586e-04,5.278456e-03,5.126019e-04,1.719011e-03,0.008708,0.040836,2.595435e-04,3.628072e-05,5.627785e-04,3.051089e-06,0.020979,5.350963e-03,2.382075e-04,4.343467e-03,9.253140e-05,4.276630e-03,6.494909e-05,4.648137e-07,0.025525,1.490522e-02,4.447343e-06,0.010642,1.674998e-03,4.171541e-05,1.597407e-05,4.585454e-03,1.585781e-02,0.039381,3.442771e-04,4.358326e-08,2.451024e-03
2,0.052695,0.027952,5.806272e-03,2.767110e-04,0.028398,1.410905e-11,0.006765,1.256883e-02,0.033054,1.407586e-02,1.143254e-02,4.597190e-07,1.277250e-02,3.935968e-05,0.022586,9.096963e-03,0.101016,6.249243e-03,2.671501e-03,2.970290e-03,1.730154e-03,3.798416e-08,1.646993e-02,0.030418,0.039018,1.933771e-11,0.013076,4.213060e-03,4.719192e-05,1.695783e-06,6.122272e-03,0.028503,1.491686e-07,2.511782e-04,7.141096e-07,0.000416,1.143596e-06,1.249541e-02,1.000968e-04,1.106399e-02,0.000846,0.033437,0.001524,1.080465e-02,1.765674e-04,2.016492e-06,2.096613e-04,0.008344,1.066915e-02,0.043351,3.355927e-09,0.048626,1.371479e-03,1.486943e-02,9.260593e-03,2.653893e-04,8.449586e-07,7.807663e-07,0.013833,6.860878e-03,3.187176e-03,8.035265e-03,1.350874e-02,4.247822e-08,7.162573e-06,4.901136e-04,6.473335e-03,2.786875e-08,1.384963e-13,2.976483e-04,0.030879,0.033470,2.385655e-05,3.999184e-04,1.261738e-06,1.205756e-05,9.796944e-03,3.772102e-09,0.014087,8.089582e-04,1.726058e-03,4.011904e-05,1.658536e-03,0.001539,0.031898,4.087583e-09,4.939

PRE-CORRECTION: Protein Zj values from every bootstrap run (columns: protein, rows: bootstrap run, displayed as fractions: 


C:\Users\Sophi\AppData\Local\Temp\ipykernel_11396\3141336667.py:18: RuntimeWarning: divide by zero encountered in true_divide
  p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 1)[0]
C:\Users\Sophi\AppData\Local\Temp\ipykernel_11396\3141336667.py:18: RuntimeWarning: invalid value encountered in true_divide
  p_zj_bootstrap_results_fraction = p_zj_bootstrap_results_absolute/np.sum(p_zj_bootstrap_results_absolute, axis = 1)[0]


,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
3,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
4,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
5,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
6,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
7,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
8,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
9,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


# Legacy stuff

## Generation of random distributions from 3 Gaussians and plotting them

In [185]:
def create_data_from_3_gaussians(mean1, mean2, mean3, std1, std2, std3, n1, n2, n3): # Creation of random data points from multiple Gaussians ki
    k1 = np.random.normal(mean1, std1, n1) #creates array with values created through Gaussian
    k2 = np.random.normal(mean2, std2, n2)
    k3 = np.random.normal(mean3, std3, n3)

    return np.concatenate([k1, k2, k3])

In [186]:
def p_xi_given_zj_from_gaussian_datapoints(kall, mean1, mean2, mean3, std1, std2, std3):
    # Calculating p_xi_given_zj (this is what Matt is working on with the simulated data)

    pdf_probability_k1 = scipy.stats.norm.pdf(kall, loc=mean1, scale=std1)
    pdf_probability_k2 = scipy.stats.norm.pdf(kall, loc=mean2, scale=std2)
    pdf_probability_k3 = scipy.stats.norm.pdf(kall, loc=mean3, scale=std3)

    p_xi_given_zj = np.vstack((pdf_probability_k1,pdf_probability_k2,pdf_probability_k3))
    p_xi_given_zj = np.transpose(p_xi_given_zj)

    print("p_xi_given_zj")
    display(pd.DataFrame(p_xi_given_zj))
    print("\n")
    
    print("likeliest peptide z of each datapoint x according to scipy.stats.norm.pdf (should be more accurate than EM because its dedicated to Gaussians)")
    print(np.argmax(p_xi_given_zj, axis=1)) # for each column of P(Xi|Zj), the most likely Peptide is returned
    print("\n")
    
    return p_xi_given_zj

In [187]:
def create_3_gaussians_and_calculate_p_xi_given_zj(mean1, mean2, mean3, std1, std2, std3, n1, n2, n3):
    kall = create_data_from_3_gaussians(mean1, mean2, mean3, std1, std2, std3, n1, n2, n3)
    p_xi_given_zj = p_xi_given_zj_from_gaussian_datapoints(kall, mean1, mean2, mean3, std1, std2, std3)
    
    return p_xi_given_zj

In [188]:
def plot_histograms_and_pdfs_from_gaussians(mean1, mean2, mean3, std1, std2, std3, n1, n2, n3):
    k1 = np.random.normal(mean1, std1, n1) #creates array with values created through Gaussian
    k2 = np.random.normal(mean2, std2, n2)
    k3 = np.random.normal(mean3, std3, n3)
    
    # plotting histograms
    nbins = 50
    plt.hist(k1, label = "Peptide 0", bins=nbins, alpha=0.3, density=True, color="orange") # alpha=transparency, density=True normalises to 1 
    plt.hist(k2, label = "Peptide 1", bins=nbins, alpha=0.3, density=True, color="green")
    plt.hist(k3, label = "Peptide 2", bins=nbins, alpha=0.3, density=True, color="blue")

    # PDF plot
    xmin, xmax = plt.xlim() #finds lower and upper bounds of histogram data
    x = np.linspace(start=xmin, stop=xmax, num=100) #num is the number of returned data points - the more points, the finer the fit is plotted
    p1 = scipy.stats.norm.pdf(x, mean1, std1)
    p2 = scipy.stats.norm.pdf(x, mean2, std2)
    p3 = scipy.stats.norm.pdf(x, mean3, std3)

    plt.plot(x, p1, linewidth=2, color = "orange", label = "Gauss function k1: mean = {:.2f}, STD = {:.2f}".format(mean1, std1))
    plt.plot(x, p2, linewidth=2, color = "green", label = "Gauss function k2: mean = {:.2f}, STD = {:.2f}".format(mean2, std2))
    plt.plot(x, p3, linewidth=2, color = "blue", label = "Gauss function k3: mean = {:.2f}, STD = {:.2f}".format(mean3, std3))

    plt.legend(loc='upper right')
    plt.title("PDFs of dwarves and humans")

    plt.show()

## EM functions WITHOUT broadcasting (slow as hell!)

In [189]:
def NOBROADCAST_update_p_zj_given_xi(p_xi_given_zj, p_zj, p_zj_given_xi):
    ## for PEPTIDE inference
    if inference_mode == "peptide" or "protein":
        denominator = 0
        for i, row in enumerate(p_xi_given_zj): # Calculating/Updating P(Zj|Xi)
            #print("ROW of P(Xi|Zj):", i)
            for j, cell in enumerate(row):
                #print("COLUMN of P(Xi|Zj):", j)
                numerator = cell * p_zj[j]
                #print("numerator:", numerator)

                for l, cell in enumerate(p_zj):
                    # print("cell", i, l, p_xi_given_zj[i, l], end="")
                    # print(" * zl", p_zj[l])
                    denominator = denominator + p_xi_given_zj[i, l] * p_zj[l]
                
                # print("p_zj_given_xi[j][i] = numerator/denominator", numerator, "/", denominator)
                #display(pd.DataFrame(p_zj_given_xi))
                p_zj_given_xi[j][i] = numerator/denominator
                denominator = 0                                
        #display(pd.DataFrame(p_zj_given_xi))    
        #print((np.argmax(p_zj_given_xi, axis=1))) # reports index of max value from each row

        
    ## for PROTEIN inference (denominator is has no loop)
    elif inference_mode == "not protein idk why this doesnt work":
        # print("p_xi_given_zj (subarray)")
        # display(pd.DataFrame(p_xi_given_zj))
        for i, row in enumerate(p_xi_given_zj): # Calculating/Updating P(Zj|Xi)#
            # print("ROW of P(Xi|Zj):", i)
            for j, cell in enumerate(row):
                # print("COLUMN of P(Xi|Zj):", j)
                # print("p_zj_given_xi", cell, "*",  p_zj[j], "/", p_xi[i],"=", cell*p_zj[j]/p_xi[i])
                p_zj_given_xi[j][i] = cell * p_zj[j]/p_xi[i]

        # display(pd.DataFrame(p_zj_given_xi))
       # print((np.argmax(p_zj_given_xi, axis=1))) # reports index of max value from each row
        
    else:
        print("ERROR: inference_mode was not set to peptide or protein")
        
    return p_zj_given_xi

In [190]:
def NOBROADCAST_update_p_zj(p_zj, p_zj_given_xi):  
    if inference_mode == "peptide":
        for j, element in enumerate(p_zj): #updating the expectation value of Zi
            p_zj[j] = p_zj_given_xi[j].sum()/p_xi_given_zj.shape[1] # divide by number of peptides z

        p_zj_sum = np.sum(p_zj, axis = 0)
        p_zj = p_zj/p_zj_sum

        # print("EM loop", EM_loopcounter, ")")
        # display(pd.DataFrame(p_zj))
    
    elif inference_mode == "protein":
        p_zj.fill(0) # overwrite all values of pzj with 0

        print("p_zj_given_xi after update")
        display(pd.DataFrame(p_zj_given_xi))
        
        for j, row in enumerate(p_zj_given_xi):            
            for i, element in enumerate(row):
                print("p_zj_given_xi[j][i] * p_xi[i]", p_zj_given_xi[j][i], p_xi[i], p_zj_given_xi[j][i] * p_xi[i])
                p_zj[j] = p_zj[j] + (p_zj_given_xi[j][i] * p_xi[i])
        
        p_zj_sum = np.sum(p_zj, axis = 0)
        p_zj = p_zj/p_zj_sum
        print("p_zj (EM loop", EM_loopcounter, ")")
        display(pd.DataFrame(p_zj))
    
    else:
        print("ERROR: inference_mode was not set to peptide or protein")
    
    return p_zj